In [1]:
import numpy as np
import statistics

ModuleNotFoundError: No module named 'numpy'

In [2]:
x = [1,2,3,4,5,2,7,2,4,4,3,6,7,2,10]
y = [0.124,0.352,0.5622,0.9897,0.23876,0.45672,0.243,0.876,0.198,0.197,0.38]

statistics.stdev(x), statistics.stdev(y)

NameError: name 'statistics' is not defined

In [12]:
np.mean(x)

0.009437307948081017